In [ ]:
pip install NERDA

     |████████████████████████████████| 2.6 MB 11.9 MB/s 
     |████████████████████████████████| 3.3 MB 60.7 MB/s 
     |████████████████████████████████| 895 kB 69.4 MB/s 
     |████████████████████████████████| 636 kB 63.4 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=2f5d9214b428b8e518f217009b30e2bca436cd98a06f01b143a86ae536c0b435
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd

from nltk import word_tokenize
from tqdm import tqdm
tqdm.pandas()

from sklearn.utils import shuffle

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_windows = "/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/combined_annotations.csv"

In [ ]:

annotations = pd.read_csv(path_windows,sep=';')
annotations = shuffle(annotations,random_state=123)
annotations.head()

,left_context,candidate_skill,right_context,label
19073,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,excellent,understanding,of the web stack,0
19817,member handbook and offer service of a,polite,efficient and professional manner,1
9569,make them easy to use,and understand,and working with ux and,0
5291,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [ ]:
candidate_skill_split = annotations['candidate_skill'].str.split(' ',expand=True)
candidate_skill_split = candidate_skill_split.add_prefix('token_')
candidate_skill_split.head()

,token_0,token_1,token_2,token_3
19073,independent,None,None,None
13367,understanding,None,None,None
19817,polite,None,None,None
9569,and,understand,None,None
5291,skills,verbal,and,None


In [ ]:
annotations = candidate_skill_split.join(annotations)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
19073,independent,None,None,None,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,understanding,None,None,None,excellent,understanding,of the web stack,0
19817,polite,None,None,None,member handbook and offer service of a,polite,efficient and professional manner,1
9569,and,understand,None,None,make them easy to use,and understand,and working with ux and,0
5291,skills,verbal,and,None,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [ ]:
annotations['token_0'] = annotations['token_0']+':'+annotations['label'].astype(str)
annotations['token_1'] = annotations['token_1']+':'+annotations['label'].astype(str)
annotations['token_2'] = annotations['token_2']+':'+annotations['label'].astype(str)
annotations['token_3'] = annotations['token_3']+':'+annotations['label'].astype(str)
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label
19073,independent:0,NaN,NaN,NaN,assist their development enable them to become...,independent,and in turn become business mentor themselves,0
13367,understanding:0,NaN,NaN,NaN,excellent,understanding,of the web stack,0
19817,polite:1,NaN,NaN,NaN,member handbook and offer service of a,polite,efficient and professional manner,1
9569,and:0,understand:0,NaN,NaN,make them easy to use,and understand,and working with ux and,0
5291,skills:0,verbal:0,and:0,NaN,the uncertain waters exceptional communication,skills verbal and,written highly organised,0


In [ ]:
annotations = annotations.fillna('')
annotations['combined'] = annotations['left_context'] + ' ' + annotations['token_0'] + ' ' + annotations['token_1'] + ' '+ annotations['token_2'] + ' ' + annotations['token_3'] + ' ' + annotations['right_context']+'.'
annotations.head()

,token_0,token_1,token_2,token_3,left_context,candidate_skill,right_context,label,combined
19073,independent:0,,,,assist their development enable them to become...,independent,and in turn become business mentor themselves,0,assist their development enable them to become...
13367,understanding:0,,,,excellent,understanding,of the web stack,0,excellent understanding:0 of the web stack.
19817,polite:1,,,,member handbook and offer service of a,polite,efficient and professional manner,1,member handbook and offer service of a polite...
9569,and:0,understand:0,,,make them easy to use,and understand,and working with ux and,0,make them easy to use and:0 understand:0 and...
5291,skills:0,verbal:0,and:0,,the uncertain waters exceptional communication,skills verbal and,written highly organised,0,the uncertain waters exceptional communication...


In [ ]:
annotations.shape[0]

20836

In [ ]:
train = annotations[['combined']].iloc[0:round(0.8*len(annotations)),]
val = annotations[['combined']].drop(train.index).iloc[0:round(0.1*len(annotations)),]
test = annotations[['combined']].drop(train.index).drop(val.index).iloc[0:round(0.1*len(annotations)),]


In [ ]:
banaan = annotations[['combined']].loc[train.index]
banaan2 = annotations[['combined']].loc[val.index]
complete_train = pd.concat([banaan,banaan2])

In [ ]:
def tuple_transformer(dataset):
    token_df = pd.DataFrame()

    dataset= dataset

    for index in tqdm(range(0,len(dataset['combined']))):
        tokens = pd.DataFrame(word_tokenize(dataset['combined'].iloc[index]),columns=['tokens'])
        tokens['index'] = index

        token_df = pd.concat([token_df,tokens])

    annotationcolumn = token_df['tokens'].str.split(':',expand=True)

    token_df['annotation'] = annotationcolumn[1]
    token_df['tokens'] = annotationcolumn[0]

    token_df['annotation'] = token_df['annotation'].fillna('O')
    token_df.annotation[token_df.annotation=='0'] = 'O'
    token_df.annotation[token_df.annotation=='1'] = 'Soft_Skill'
    token_df.annotation[token_df.annotation=='2'] = 'Hard_Skill'

    token_df = token_df.rename(columns={'index':'sentence'})

    token_df['tuples'] = list(zip(token_df.tokens, token_df.annotation))

    tuple_list = []
    sublist=[]
    for row in tqdm(token_df['tuples']):
        sublist.append(row)
        if row[0]=='.':
            tuple_list.append(sublist)
            sublist=[]
            next

    return tuple_list

In [ ]:
import nltk
nltk.download('punkt')
training = tuple_transformer(train)
validation = tuple_transformer(val)
testing = tuple_transformer(test)
cross_val_data = tuple_transformer(complete_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 16669/16669 [00:40<00:00, 408.68it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 203428/203428 [00:00<00:00, 1738741.84it

In [ ]:

def list_of_lists(dataset):
    texts = []
    tags = []

    token_sublist = []
    label_sublist = []

    for sublist in tqdm(dataset):
        for token,label in sublist:

            token_sublist.append(token)
            label_sublist.append(label)
            if token == '.':            
                texts.append(token_sublist)
                tags.append(label_sublist)

                token_sublist = []
                label_sublist = []
    return texts, tags

training_text, training_label = list_of_lists(training)
test_text, test_label = list_of_lists(testing)
val_text, val_label = list_of_lists(validation)
cross_text, cross_label = list_of_lists(cross_val_data)

100%|██████████| 18753/18753 [00:00<00:00, 281920.37it/s]


In [ ]:
def dict_creator(text,label):
    from collections import defaultdict
    my_dict = defaultdict(list)
    for x in text:
        my_dict['sentences'].append(x)

    for x in label:
        my_dict['tags'].append(x)
    
    return dict(my_dict)

In [ ]:
training = dict_creator(training_text, training_label)
validation = dict_creator(val_text, val_label)
testing = dict_creator(test_text, test_label)
cross_training = dict_creator(cross_text, cross_label)

In [ ]:
tag_scheme = [
#              'O',
'Soft_Skill',
'Hard_Skill'
]

In [ ]:
transformer = 'bert-base-uncased'

In [ ]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 3,
'warmup_steps' : 500,                                                   
    'train_batch_size': 13,                                         
    'learning_rate': 0.0001
}




In [ ]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = training,
dataset_validation = validation,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = transformer,
dropout = dropout,
hyperparameters = training_hyperparameters
)

Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.train()

  0%|          | 0/1283 [00:00<?, ?it/s]


 Epoch 1 / 3


  0%|          | 0/1283 [00:00<?, ?it/s]

Train Loss = 0.10725680326052817 Valid Loss = 0.06478854549078135

 Epoch 2 / 3


  0%|          | 0/1283 [00:00<?, ?it/s]

Train Loss = 0.049652043406884075 Valid Loss = 0.0636003717072143

 Epoch 3 / 3


100%|██████████| 261/261 [00:18<00:00, 14.11it/s]

Train Loss = 0.026240993840295764 Valid Loss = 0.07004632841780295


'Model trained successfully'

In [ ]:
emscad = pd.read_csv('/content/drive/MyDrive/NERDA learning/emscad_anonymized.csv')

In [ ]:
emscad['job_description'] = emscad['description'] + ' ' + emscad['requirements'] 
emscad = emscad.dropna(axis=0, subset=['job_description'])
emscad.shape

(15187, 19)

In [ ]:

index = []
tokens = []
tags = []

for x in tqdm(range(0,len(emscad['job_description']))):

  predictions = model.predict_text(emscad['job_description'].iloc[x])

  tokens_list = [b for x in predictions[0] for b in x]
  labels_list = [b for x in predictions[1] for b in x]
    
  tokens.append(tokens_list)
  tags.append(labels_list)

    
    
    

Streaminguitvoer ingekort tot de laatste 5000 regels.


 91%|█████████ | 13774/15187 [1:06:50<09:25,  2.50it/s]


 91%|█████████ | 13775/15187 [1:06:51<09:10,  2.57it/s]


 91%|█████████ | 13776/15187 [1:06:51<09:06,  2.58it/s]


 91%|█████████ | 13777/15187 [1:06:51<08:24,  2.79it/s]


 91%|█████████ | 13778/15187 [1:06:52<08:29,  2.77it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #15 length 189 exceeds max_len 128 and has been truncated
  warnings.warn(msg)



 91%|█████████ | 13779/15187 [1:06:52<08:44,  2.68it/s]


 91%|█████████ | 13780/15187 [1:06:53<08:07,  2.89it/s]


 91%|█████████ | 13781/15187 [1:06:53<08:00,  2.92it/s]


 91%|█████████ | 13782/15187 [1:06:53<08:27,  2.77it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #9 length 151 exceeds max_len 128 and has been truncated
  warnings.warn(msg)



 91%|█████████ | 13783/15187 [1:06:54<08:21,  2.80it/s]


 91%|█████████ | 13784/15187 [1

In [ ]:
all_tokens = [b for x in tokens for b in x]
all_labels = [b for c in tags for b in c]

all_predictions = pd.DataFrame(zip(all_tokens,all_labels), columns=['token','label'])
all_predictions
all_predictions.to_csv('/content/drive/MyDrive/NERDA learning/predictions.csv')

In [ ]:
 tokens = [b for x in predictions[0] for b in x]
 labels = [b for x in predictions[1] for b in x]
 pd.DataFrame(zip(tokens,labels))

,0,1
0,Food,O
1,",",O
2,a,O
3,fast-growing,O
4,",",O
...,...,...
255,dishes,O
256,",",O
257,shopping,O
258,",",O


In [ ]:
#Cross validation NERDA model
from sklearn.model_selection import KFold
import numpy as np
folds = 10
kf = KFold(n_splits=folds)
kf.get_n_splits(cross_text)

from sklearn.metrics import precision_score, recall_score, f1_score

from itertools import chain



preds = []
precision = []
recall = []
f1 = []

for train_index, val_index in kf.split(cross_text):

  train_K = dict_creator(np.array(cross_text)[train_index], np.array(cross_label)[train_index])
  val_K = dict_creator(np.array(cross_text)[val_index], np.array(cross_label)[val_index])

  model = NERDA(
  dataset_training = train_K,
  dataset_validation = val_K,
  tag_scheme = tag_scheme, 
  tag_outside = 'O',
  transformer = transformer,
  dropout = dropout,
  hyperparameters = training_hyperparameters
  )
  model.train()

  predictions = model.predict(test_text)

  predictions2 = list(chain.from_iterable(predictions))
  test_label2 = list(chain.from_iterable(test_label))
  preds.append(predictions)


  precision.append(precision_score(test_label2,predictions2,average='macro'))
  recall.append(recall_score(test_label2,predictions2,average='macro'))
  f1.append(f1_score(test_label2,predictions2,average='macro'))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/1299 [00:00<?, ?it/s]


 Epoch 1 / 3


  0%|          | 0/1299 [00:00<?, ?it/s]

...

Train Loss = 0.10793729224846303 Valid Loss = 0.07278627332775517

 Epoch 2 / 3


  0%|          | 0/1299 [00:00<?, ?it/s]

Train Loss = 0.05046527133835605 Valid Loss = 0.0631673306507861

 Epoch 3 / 3


100%|██████████| 235/235 [00:16<00:00, 14.55it/s]

Train Loss = 0.02726810257219034 Valid Loss = 0.0665737747571566


In [ ]:
from statistics import mean

print('precision score', mean(precision))

print('recall score', mean(recall))

print('f1 score', mean(f1))

precision score 0.756870799185615
recall score 0.740533766102767
f1 score 0.7481155728158868
